In [ ]:
#Importing save state folder from drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Creating a Binary Classifier Discriminator:

In [19]:
#Importing necessary modules
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

#Filling in the Discriminator Class Structure for given scenario
class Discriminator(object):
    
    def __init__(self, width=28, height=28, channels=1, latent_size=100):
        
        #Creating internal variables
        self.CAPACITY = width * height * channels #Computing the total capacity of the model
        self.SHAPE = (width, height, channels) #Computing the shape of data for the model
        self.OPTIMIZER = Adam(lr=0.0002, decay=8e-9) #Creating ADAM optimizer for the model
        
        #Initializing the Discriminator model
        self.Discriminator = self.model()
        
        #Compiling the model for binary crossentropy loss as it is a binary classifier
        self.Discriminator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER, metrics=['accuracy'])
        
        #Displaying summary of the Discriminator model
        self.Discriminator.summary()
        
    def model(self):
        
        #Instantiating a Sequential Model for binary classification task
        model = Sequential()
        
        #Adding the first layer to the network 
        model.add(Flatten(input_shape = self.SHAPE)) #The layer flattens the data into a single data stream
        
        #Adding the next set of layers to the network
        model.add(Dense(self.CAPACITY, input_shape=self.SHAPE)) #The dense layer is fully connected with next layer
        model.add(LeakyReLU(alpha=0.2)) #Leaky ReLU layer ensures use of small gradients
        
        #Adding the next set of layers to the network
        model.add(Dense(int(self.CAPACITY/2)))
        model.add(LeakyReLU(alpha=0.2))
        
        #Adding the output layer to the network
        model.add(Dense(1, activation='sigmoid'))
        return model
    
    def summary(self):
        
        #Returning summary of the Discriminator model
        return self.Discriminator.summary()
    
    def save_model(self):
        
        #Plotting photographic version of model structure
        plot_model(self.Discriminator.model, to_file="/content/drive/Data/Discriminator_Model.png")

## Creating Simple Generator Model:

In [20]:
#Importing necessary modules
import sys
import numpy as np
from keras.layers import Dense, Reshape
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

#Filling in the Generator Base Class for given scenario
class Generator(object):
    
    #Initialization Method
    def __init__(self, width=28, height=28, channels=1, latent_size=100):
        
        #Creating internal variables
        self.W = width #Storing the width of image
        self.H = height #Storing the height of image
        self.C = channels #Storing the channels for the image
        self.OPTIMIZER = Adam(lr=0.002, decay=8e-9) #Creating Adam optimizer for model development
        
        #Storing the latent space size for the generator
        self.LATENT_SPACE_SIZE = latent_size
        
        #Creating a latent space for the generator model via a Gaussian distribution
        self.latent_space = np.random.normal(0, 1, (self.LATENT_SPACE_SIZE,))
        
        #Instantiating the generator model object
        self.Generator = self.model()
        
        #Compiling the generator model with binary crossentropy loss and Adam optimizer
        self.Generator.compile(loss="binary_crossentropy", optimizer=self.OPTIMIZER)
        
        #Displaying the summary of the generator model
        self.Generator.summary()
    
    #Model Instantiation Method
    def model(self, block_starting_size=128, num_blocks=4):
        
        #Instantiating a Sequential Model for Generator model
        model = Sequential()
        
        #Storing the initialized block size for number of nodes 
        block_size = block_starting_size
        
        #Generate the first block of layers for the network
        model.add(Dense(block_size, input_shape=(self.LATENT_SPACE_SIZE,))) #Creates a basic input Dense interconnected layer
        model.add(LeakyReLU(alpha=0.2)) #Creates a LeakyReLU layer to prevent vanishing gradients and non-activated neurons
        model.add(BatchNormalization(momentum=0.8)) #Performs BatchNormalization to clean layer by normalizing activations
    
        #Adding the remaining blocks of layers with double the block size with each block
        for i in range(num_blocks - 1):
            block_size = block_size * 2
            model.add(Dense(block_size))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            
        #Reconstructing output to the input image shape
        model.add(Dense(self.W * self.H * self.C, activation='tanh'))
        model.add(Reshape((self.W, self.H, self.C)))
        
        #Returning the model
        return model
        
    #Summarization Method
    def summary(self):
        #Produces summary of the model on the screen
        return self.Generator.summary()
    
    #Saving Method
    def save_model(self):
        #Saves the model structure to a file in particular folder
        plot_model(self.Generator.model, to_file='/content/drive/Data/Generator_Model.png')

## Creating the Adversarial Network:

In [21]:
#Creating GAN Base Class Structure
class GAN(object):
    
    #Initialization Method
    def __init__(self, discriminator, generator):
        self.OPTIMIZER = Adam(lr=0.002, decay=8e-9) #Creating an Adam optimizer
        self.Generator = generator #Storing the provided generator model
        self.Discriminator = discriminator #Storing the provided discriminator model
        
        self.Discriminator.trainable = False #Trainability for discriminator turned off for adversarial training
        self.gan_model = self.model() #Creating the adversarial model
        self.gan_model.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER) #Compiling adversarial model
        self.gan_model.summary() #Displaying summary of the adversarial model
    
    #Model Instantiation Method
    def model(self):
        model = Sequential() #Creating a Sequential model
        model.add(self.Generator) #Adding the generator model to the network
        model.add(self.Discriminator) #Adding the discriminator model to the network
        return model
    
    #Summarization Method
    def summary(self):
        #Produces summary of the model on the screen
        self.gan_model.summary()
    
    #Saving Method
    def save_model(self):
        #Saves the model structure to file in a particular folder
        plot_model(self.gan_model.model, to_file='/content/drive/Data/GAN_Model.png')

## Training GAN Model:

In [28]:
#Importing the necessary modules
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from random import randint

#Creating the Trainer class to train the adversarial network
class Trainer:
    
    def __init__(self, width=28, height=28, channels=1, latent_size=100, epochs=50000, batch=32, checkpoint=50, model_type=-1):
        #Creating internal variables
        self.W = width #Storing the width of the data
        self.H = height #Storing the height of the data
        self.C = channels #Storing the channels of the data
        self.EPOCHS = epochs #Storing the number of epochs for the data
        self.BATCH = batch #Storing the number of batches for the data
        self.CHECKPOINT = checkpoint #Storing the checkpoint for the data
        self.model_type = model_type #Storing the model type for the model
        self.LATENT_SPACE_SIZE = latent_size #Storing the latent size for the image
        
        #Creating Generator model for training
        self.generator = Generator(height=self.H, width=self.W, channels=self.C, latent_size=self.LATENT_SPACE_SIZE)
        
        #Creating Discriminator model for training
        self.discriminator = Discriminator(height=self.H, width=self.W, channels=self.C)
        
        #Creating Adversarial model for training
        self.gan = GAN(generator=self.generator.Generator, discriminator=self.discriminator.Discriminator)
        
        #Loading the mnist data for each Trainer instance
        self.load_MNIST()
        
    #Creating the dataset loading method
    def load_MNIST(self, model_type=3):
        allowed_types = list(range(-1,10)) #Creating list of allowed model types
        if self.model_type not in allowed_types:
            print("ERROR! Only integer values ranging within the interval [-1, 9] are allowed!")
            
        #Loading the training and testing data
        (self.X_train, self.Y_train), (_, _) = mnist.load_data()
        
        #Checking model type (if it is for a specific output or all outputs) 
        if self.model_type != -1:
            self.X_train = self.X_train[np.where(self.Y_train == int(self.model_type))[0]]
            
        #If model type is -1 (for all outputs)
        self.X_train = (np.float32(self.X_train) - 127.5) / 127.5
        self.X_train = np.expand_dims(self.X_train, axis=3)
        
        return
    
    #Creating the model training method
    def train(self):
        #Iterating for each epoch of training set
        for e in range(self.EPOCHS):
            #Grabbing a batch of images (half of the images)
            count_real_images = int(self.BATCH / 2)
            
            #Create a random index for the batch
            starting_index = randint(0, len(self.X_train) - count_real_images)
            
            #Extracting the half the number of images based on the starting index
            real_images_raw = self.X_train[starting_index : (starting_index + count_real_images)]
            
            #Creating real images and labels for training the batch data
            x_real_images = real_images_raw.reshape(count_real_images, self.W, self.H, self.C)
            y_real_labels = np.ones([count_real_images, 1])
            
            #Creating latent space for the images to be generated
            latent_space_samples = self.sample_latent_space(count_real_images)
            
            #Generating images for the batch of images via the Generator model
            x_generated_images = self.generator.Generator.predict(latent_space_samples)
            
            #Generating labels for the generated images
            y_generated_labels = np.zeros([self.BATCH - count_real_images, 1])
            
            #Concatenating both the batches (real images and generated ones) for training on discriminator
            x_batch = np.concatenate([x_real_images, x_generated_images])
            y_batch = np.concatenate([y_real_labels, y_generated_labels])
            
            #Training the Discriminator on the batch
            discriminator_loss = self.discriminator.Discriminator.train_on_batch(x_batch, y_batch)[0]
            
            #Generating noise to train Generator model
            x_latent_space_samples = self.sample_latent_space(self.BATCH)
            y_generated_labels = np.ones([self.BATCH, 1])
            
            #Training the Generator on the batch
            generator_loss = self.gan.gan_model.train_on_batch(x_latent_space_samples, y_generated_labels)
            
            #Viewing the loss metrics for the batch
            print("Epoch: {} , [Discriminator :: Loss : {}] , [Generator :: Loss : {}]".format(e, discriminator_loss, generator_loss))
            
            #Saving checkpoint 
            if e % self.CHECKPOINT == 0:
                self.plot_checkpoint(e)
            
        return
    
    #Helper method : A wrapper method to produce latent space easily
    def sample_latent_space(self, instances):
        return np.random.normal(0, 1, (instances, self.LATENT_SPACE_SIZE))
    
    #Helper method: Plotting checkpoint on reaching checkpoint epoch
    def plot_checkpoint(self, e):
        #Generating filename to save to
        filename = "/content/drive/Data/samples/" + str(e) + ".png"
        
        #Creating noise from latent space and generating an image via Generator
        noise = self.sample_latent_space(16)
        images = self.generator.Generator.predict(noise)
        
        #Plotting the newly generated images at each epoch checkpoint
        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1) #Creating 4x4 grid for 16 generated images
            image = images[i, :, :, :]
            image = np.reshape(image, (self.H, self.W))
            plt.imshow(image, cmap='gray')
            plt.axis('off')
            
            plt.tight_layout()
            #plt.savefig(filename)
            plt.close('all')
            
        return

In [29]:
#Running the Trainer class to Train the adversarial network
#Defining internal variable values
HEIGHT = 28
WIDTH = 28
CHANNEL = 1
LATENT_SPACE_SIZE = 100
EPOCHS = 500
BATCH = 4
CHECKPOINT = 50
MODEL_TYPE = -1

#Creating the trainer object
trainer = Trainer(height=HEIGHT, width=WIDTH, channels=CHANNEL, latent_size=LATENT_SPACE_SIZE, epochs=EPOCHS, batch=BATCH ,checkpoint=CHECKPOINT, model_type=MODEL_TYPE)

#Training the trainer object to get results
trainer.train()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 128)               12928     
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 128)               512       
_________________________________________________________________
dense_49 (Dense)             (None, 256)               33024     
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 256)               1024      
_________________________________________________________________
dense_50 (Dense)             (None, 512)             